In [1]:
import re
from io import StringIO
from datetime import datetime, timedelta,date

import requests
import pandas as pd


class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://in.finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'
    

    def __init__(self, symbol):
        f_date=date(1994,1,1)
        t_date=date.today()
        delta=t_date-f_date
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=delta.days)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])


In [2]:
df = YahooFinanceHistory('SBIN.NS').get_quote()

In [3]:
df_tcs=YahooFinanceHistory('TCS.NS').get_quote()

In [4]:
df_tcs

Date         Open         High          Low        Close  \
0    2004-08-25   149.837006   149.837006   122.375000   123.494003   
1    2004-08-26   124.000000   124.625000   121.912003   122.375000   
2    2004-08-27   122.800003   122.800003   119.820000   120.332001   
3    2004-08-30   121.237999   123.750000   120.625000   123.345001   
4    2004-08-31   123.311996   123.750000   122.000000   123.512001   
5    2004-09-01   123.750000   124.375000   122.949997   123.487999   
6    2004-09-02   123.737999   125.574997   123.250000   124.207001   
7    2004-09-03   125.750000   137.500000   123.794998   124.732002   
8    2004-09-06   129.988007   129.988007   124.112999   124.357002   
9    2004-09-07   129.375000   129.375000   124.375000   124.449997   
10   2004-09-08   124.500000   125.099998   123.887001   124.212997   
11   2004-09-09   124.625000   124.737999   122.307999   122.495003   
12   2004-09-10   123.750000   123.750000   122.000000   123.599998   
13   2004-09-13   123.875000   126.938004   123.875000   125.438004   
14   2004-09-14   125.625000   127.349998   125.370003   126.957001   
15   2004-09-15   127.250000   127.500000   125.150002   125.762001   
16   2004-09-16   125.875000   126.875000   125.311996   126.057999   
17   2004-09-17   126.500000   128.738007   126.287003   128.061996   
18   2004-09-20   129.050003   129.613007   127.519997   127.862999   
19   2004-09-21   128.106995   131.061996   127.750000   130.725006   
20   2004-09-22   131.050003   132.000000   129.636993   131.507996   
21   2004-09-23   131.000000   131.125000   128.375000   128.783005   
22   2004-09-24   128.636993   130.000000   128.250000   128.600006   
23   2004-09-27   128.438004   129.061996   126.875000   127.595001   
24   2004-09-28   127.000000   128.425003   126.561996   126.945000   
25   2004-09-29   127.407997   129.125000   127.019997   128.875000   
26   2004-09-30   129.449997   130.113007   127.582001   128.386993   
27   2004-10-01   128.625000   132.057007   128.294998   130.962006   
28   2004-10-04   132.125000   135.125000   131.807007   134.561996   
29   2004-10-05   135.037994   136.125000   134.438004   135.087006   
...         ...          ...          ...          ...          ...   
3679 2019-07-09  2174.000000  2174.399902  2104.550049  2133.350098   
3680 2019-07-10  2080.000000  2127.850098  2071.300049  2108.199951   
3681 2019-07-11  2128.000000  2129.000000  2086.050049  2102.550049   
3682 2019-07-12  2102.100098  2119.750000  2093.100098  2107.600098   
3683 2019-07-15  2125.000000  2153.600098  2098.000000  2145.699951   
3684 2019-07-16  2126.000000  2132.500000  2100.000000  2106.000000   
3685 2019-07-17  2108.000000  2125.000000  2092.100098  2117.250000   
3686 2019-07-18  2116.199951  2116.199951  2060.000000  2065.949951   
3687 2019-07-19  2077.000000  2097.000000  2062.500000  2076.949951   
3688 2019-07-22  2077.000000  2117.449951  2068.800049  2109.899902   
3689 2019-07-23  2127.899902  2130.000000  2102.399902  2112.449951   
3690 2019-07-24  2110.000000  2120.800049  2076.500000  2096.800049   
3691 2019-07-25  2108.899902  2135.000000  2092.550049  2127.899902   
3692 2019-07-26  2129.800049  2129.800049  2095.000000  2109.050049   
3693 2019-07-29  2110.800049  2142.000000  2103.300049  2130.000000   
3694 2019-07-30  2130.000000  2185.000000  2120.449951  2179.149902   
3695 2019-07-31  2171.750000  2211.699951  2147.149902  2205.699951   
3696 2019-08-01  2200.000000  2217.500000  2166.250000  2180.050049   
3697 2019-08-02  2173.000000  2210.000000  2165.500000  2205.300049   
3698 2019-08-05  2199.899902  2252.250000  2195.600098  2248.600098   
3699 2019-08-06  2232.800049  2251.949951  2211.449951  2214.899902   
3700 2019-08-07  2230.000000  2238.500000  2207.250000  2213.449951   
3701 2019-08-08  2228.899902  2281.000000  2218.899902  2258.100098   
3702 2019-08-09  2260.000000  2267.899902  2241.350098  2246.250000   
3703 2019-08-13  2245.000000  2246.

In [5]:
df_tcs=YahooFinanceHistory('^NSEI').get_quote()

In [6]:
df_tcs.tail()

Date          Open          High           Low         Close  \
2937 2019-08-13  11139.400391  11145.900391  10901.599609  10925.849609   
2938 2019-08-14  11003.250000  11077.349609  10935.849609  11029.400391   
2939 2019-08-16  11043.650391  11068.549805  10924.299805  11047.799805   
2940 2019-08-19  11094.799805  11146.900391  11037.849609  11053.900391   
2941 2019-08-21  11018.150391  11034.200195  10906.650391  10918.700195   

         Adj Close  Volume  
2937  10925.849609   600.0  
2938  11029.400391   500.0  
2939  11047.799805   500.0  
2940  11053.900391   400.0  
2941  10918.700195     0.0

In [7]:
df['Adj Close']

0         0.442401
1         0.428321
2         0.416900
3         0.415460
4         0.413131
5         0.401045
6         0.423000
7         0.403595
8         0.419006
9         0.419230
10        0.420560
11        0.429430
12        0.411800
13        0.409027
14        0.413906
15        0.428321
16        0.432866
17        0.418231
18        0.422221
19             NaN
20        0.441515
21        0.451384
22        0.435971
23        0.457591
24        0.489193
25        0.520017
26        0.524452
27        0.515249
28        0.529884
29        0.567362
           ...    
5936    359.500000
5937    354.200012
5938    363.200012
5939    363.600006
5940    360.049988
5941    364.350006
5942    372.399994
5943    363.649994
5944    356.000000
5945    350.850006
5946    342.200012
5947    339.600006
5948    341.299988
5949    342.600006
5950    343.799988
5951    327.549988
5952    332.200012
5953    317.149994
5954    308.450012
5955    300.250000
5956    301.399994
5957    289.